In [2]:
import builtins
import copy

from selenium import webdriver
from selenium.common import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service
from itertools import combinations
from datetime import datetime

import time

In [3]:
def print(*args, **kwargs):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    builtins.print(f"[{timestamp}] ", *args, **kwargs)

In [4]:
EDGE_DRIVER_PATH = "edgedriver_win32/msedgedriver.exe"
driver = webdriver.Edge(service=Service(EDGE_DRIVER_PATH))

In [5]:
wait = WebDriverWait(driver, 10)

In [6]:
driver.get("https://rustypot.com/coinflip")

In [101]:
def get_possible_bet_values(inventory_items_values, max_items=4):
    print(f"there are {len(inventory_items_values)} items in inventory with total value: {sum(inventory_items_values):.2f}$")
    possible_bet_values = {}
    for r in range(1, max_items + 1):
        for combo in combinations(inventory_items_values, r):
            s = sum(combo)
            max_tax = s * 2 * 0.05
            good_combo = True
            for item_value in combo:
                if item_value < max_tax:
                    good_combo = False
            if good_combo:
                possible_bet_values[s] = list(combo)
    possible_bet_values_sorted = dict(sorted(possible_bet_values.items()))
    
    return possible_bet_values_sorted

In [98]:
def get_inventory_items():
    create_a_game_buttom = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "createCoinflipButton"))
    )
    create_a_game_buttom.click()
    
    try:
        inventory_items = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "InventoryItem"))
        )
        
        # inventory_items = inventory_items_div.find_elements(By.CLASS_NAME, "InventoryItem")
        inventory_items_values = []
        for inventory_item in inventory_items:
            inventory_items_values.append(float(inventory_item.text.split("$")[1]))
        
        driver.get("https://rustypot.com/coinflip")

        print(f"inventory items gathered: {inventory_items_values}", )
        
        return inventory_items_values
    except TimeoutException:
        print(f"Timed out waiting for inventory items")
        driver.get("https://rustypot.com/coinflip")
        return get_inventory_items()

In [37]:
inventory_items_values = get_inventory_items()
# inventory_items_values = [62.31, 31.75, 31.75, 20.86, 17.38, 13.34, 13.34] ### TEST WITH HARDCODED ITEMS
possible_bet_values = get_possible_bet_values(inventory_items_values)
possible_bet_values

[2025-03-20 13:33:53]  inventory value: 271.68
inventory items gathered: [61.15, 20.38, 15.14, 15.0, 12.99, 11.5, 11.5, 10.7, 10.32, 10.32, 10.17, 8.19, 8.19, 7.03, 6.84, 6.08, 5.51, 4.9, 4.77, 4.29, 4.23, 3.91, 3.85, 3.74, 2.39, 1.46, 1.22, 1.15, 1.05, 0.97, 0.8, 0.73, 0.61, 0.6]
[2025-03-20 13:33:53]  there are 34 items in inventory


{0.6: [0.6],
 0.61: [0.61],
 0.73: [0.73],
 0.8: [0.8],
 0.97: [0.97],
 1.05: [1.05],
 1.15: [1.15],
 1.21: [0.61, 0.6],
 1.22: [1.22],
 1.33: [0.73, 0.6],
 1.3399999999999999: [0.73, 0.61],
 1.4: [0.8, 0.6],
 1.4100000000000001: [0.8, 0.61],
 1.46: [1.46],
 1.53: [0.8, 0.73],
 1.5699999999999998: [0.97, 0.6],
 1.58: [0.97, 0.61],
 1.65: [1.05, 0.6],
 1.6600000000000001: [1.05, 0.61],
 1.7: [0.97, 0.73],
 1.75: [1.15, 0.6],
 1.7599999999999998: [1.15, 0.61],
 1.77: [0.97, 0.8],
 1.78: [1.05, 0.73],
 1.8199999999999998: [1.22, 0.6],
 1.83: [1.22, 0.61],
 1.85: [1.05, 0.8],
 1.88: [1.15, 0.73],
 1.95: [1.15, 0.8],
 2.02: [1.05, 0.97],
 2.06: [1.46, 0.6],
 2.07: [1.46, 0.61],
 2.12: [1.15, 0.97],
 2.19: [1.22, 0.97],
 2.2: [1.15, 1.05],
 2.26: [1.46, 0.8],
 2.27: [1.22, 1.05],
 2.37: [1.22, 1.15],
 2.39: [2.39],
 2.4299999999999997: [1.46, 0.97],
 2.51: [1.46, 1.05],
 2.61: [1.46, 1.15],
 2.6799999999999997: [1.46, 1.22],
 2.99: [2.39, 0.6],
 3.0: [2.39, 0.61],
 3.12: [2.39, 0.73],
 3.190

In [107]:
def get_bet_items(bet_values, coinflip_value):
    for bet_value in bet_values:
        if (coinflip_value * 0.9 < bet_value < coinflip_value * 0.92 and 
                coinflip_value not in bets and 
                min_bet_value < coinflip_value < max_bet_value):
            return bet_values[bet_value]
    
    return None


def join_coinflip(join_button):

    try:
        join_button.click()
        
        spinner_locator = (By.CLASS_NAME, "spinner")
        WebDriverWait(driver, 5).until(
            EC.invisibility_of_element_located(spinner_locator)
        )
    except TimeoutException:
        print(f"Inventory loading timed out during selecting bet items")
        driver.get("https://rustypot.com/coinflip")
        join_coinflip(join_button)
    
    
def select_bet_items(bet_items_values):
    inventory_items_locator = (By.CLASS_NAME, "InventoryItem")
    inventory_items = WebDriverWait(driver, 3).until(
        EC.presence_of_all_elements_located(inventory_items_locator)
    )
    
    for inventory_item in inventory_items:
        item_value = float(inventory_item.text.split("$")[1])
        if item_value in bet_items_values:
            inventory_item.click()
            bet_items_values.remove(item_value)
    

def press_deposit_button_and_accept():
    deposit_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//button[@onclick='requestCfDeposit()']"))
    )
    deposit_button.click()
    ### SELECT THE ITEMS ###
    
    ### DEPOSIT AND CLOSE STEAM WINDOW ###
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[@onclick='closeAcceptDeposit()']"))
    )
    accept_button.click()


def accept_steam_trade():
    
    # 1.open the new window
    WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
                
    original_window = driver.current_window_handle
    new_window = [w for w in driver.window_handles if w != original_window][0]
    driver.switch_to.window(new_window)
    
    
    # 2.close if profile reported    
    try:
        warning_element = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((
                By.XPATH,
                "//*[contains(., 'have reported Blown for attempting trade scams')]"
            ))
        )
        ok_button = WebDriverWait(driver, 1).until(
            EC.element_to_be_clickable((
                By.XPATH,
                "//div[contains(@class, 'btn_grey_steamui') and span[text()='OK']]"
                ))
        )
        ok_button.click()
    except TimeoutException:
        pass
    
    # 3. Accept the trade
    confirm_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'content') and contains(text(), 'Confirm trade contents')]"))
    )
    confirm_button.click()
    time.sleep(0.25)
    
    
    confirm_gift_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[span[text()='Yes, this is a gift']]"))
    )
    confirm_gift_button.click()
    time.sleep(0.25)

    
    
    accept_trade_btn = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "trade_confirmbtn"))
    )
    accept_trade_btn.click()
    time.sleep(0.25)

    
    driver.close()
    driver.switch_to.window(original_window)
    
    
def open_cf_window_and_wait_to_flip():
    view_button = coinflip_div.find_element(By.XPATH, ".//button[text()='View']")
    view_button.click()
    
    # Wait until the <p> inside #fliper-coin disappears
    try:
        p_element = driver.find_element(By.CSS_SELECTOR, "#fliper-coin > p")
        WebDriverWait(driver, 120).until(EC.staleness_of(p_element))
    except:
        # If <p> is already gone, just wait for the new state
        pass
        
    
    WebDriverWait(driver, 120).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#fliper-coin .flipper"))
    )


def get_winner():
    # Wait for the parent div to appear
    creator_div = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "creator-imgs")))
    
    # Wait for the child img.pick within that div
    color_img = WebDriverWait(driver, 10).until(lambda d: creator_div.find_element(By.XPATH, ".//img[@class='pick']"))
    
    # Get the src
    src = color_img.get_attribute("src")
    
     # Extract the color from the filename
    if "Red" in src:
        creator_color = "Red"
    elif "Black" in src:
        creator_color = "Black"
    else:  # creator_color = "Unknown"
        time.sleep(0.5)
        print(f"creator_color is {src}. Retry getting creator")
        return get_winner()
    
    # 1. Get the 'flipper' element
    flipper = driver.find_element(By.CLASS_NAME, "flipper")
    
    # 2. Get the 'transform' style attribute
    transform_style = flipper.get_attribute("style")  # e.g. "transform: rotateY(1620deg);"
    
    # 3. Extract the degree
    import re
    match = re.search(r'rotateY\((\d+)deg\)', transform_style)
    if match:
        degrees = int(match.group(1))
        # Normalize the rotation within 360°
        final_angle = degrees % 360
    
        # Every 180° flips the face, so even number of 180s = same side
        flips = (degrees // 180) % 2
    
        flip_face = "Red" if flips == 1 else "Black"
        print("Coin landed on:", flip_face)
    else:
        time.sleep(0.5)
        print("Couldn't determine coin face. Retry getting flipping face")
        return get_winner()
        
    return creator_color, flip_face

In [70]:
tax_procentage = 0.05
min_bet_value = 1.5
max_bet_value = 5.0

bets = []

In [ ]:
from selenium.common import StaleElementReferenceException

inventory_items_values = get_inventory_items()
possible_bet_values = get_possible_bet_values(inventory_items_values)

    
while True:

    active_coinflips = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ActiveCoinflips")))
    coinflip_row_divs = active_coinflips.find_elements(By.CLASS_NAME, "coinflip")
    
    for coinflip_div in coinflip_row_divs:
        try:
            coinflip_value = float(coinflip_div.get_attribute("coinflip-value"))
            coinflip_items = coinflip_div.find_element(By.CLASS_NAME, "coinflipGameItems")
        except StaleElementReferenceException:
            print(f"coinflip divs are stale. Retry getting coinflip rows")
            break
            
        if "items" in coinflip_items.text:
            # print("more than 5 items...")
            continue
            
        img_elements = coinflip_items.find_elements(By.TAG_NAME, "img")
        
        coinflip_items_values = []
        for img in img_elements:
            title = img.get_attribute("data-original-title")
            if "|" in title:
                price_str = title.split("$")[-1]
                price = float(price_str)
                coinflip_items_values.append(price)
    
        tax_value = coinflip_value * 2 * tax_procentage  # the taxed amount is the total amount of the coinflip
        
        if any(tax_value > item_value for item_value in coinflip_items_values):
            # print(f"not profitable")
            continue
        
        
        # to join a cf, i need to deposit minimum value
        # 1. get the bet items
        bet_items_values = get_bet_items(possible_bet_values, coinflip_value)
        
        if bet_items_values is None:
            # no available bet for this cf based on our items
            continue
        
        ### Press join on available coinflip. if it is ongoing (only View button visible, skip)
        join_button = coinflip_div.find_element(By.TAG_NAME, "button")
        if "Join" not in join_button.text:
            continue
            
        print(f"bet_value = {sum(bet_items_values):.2f} -> bet_items_value{bet_items_values=}")

        join_coinflip(join_button)
        
        try:
            select_bet_items(copy.deepcopy(bet_items_values))
        except TimeoutException:
            print(f"Timed out when selecting bet items")
            driver.get("https://rustypot.com/coinflip")
            time.sleep(0.5)
            break
            
        press_deposit_button_and_accept()
        
        accept_steam_trade()

 
        print(f"{coinflip_value=} -> {coinflip_items_values=}")
            
        bets.append(coinflip_value)
        
        open_cf_window_and_wait_to_flip()

        creator_color, flip_face = get_winner()

        # extract flip color
        
        if creator_color == flip_face:
            print(f"flip was lost!")
            time.sleep(4) # wait to see the flip

            driver.get("https://rustypot.com/coinflip")
            time.sleep(1)
            for item in bet_items_values:
                inventory_items_values.remove(item)

            possible_bet_values = get_possible_bet_values(inventory_items_values)
            
        else:
            print(f"flip was won!")
            time.sleep(20) # TODO: implement accept winnings
            
            for item in coinflip_items_values:
                inventory_items_values.append(item)

            possible_bet_values = get_possible_bet_values(inventory_items_values)

        
        print(f"Back to searching")
    time.sleep(1)

[2025-03-20 17:55:35]  inventory items gathered: [61.3, 20.34, 15.13, 15.0, 12.96, 11.41, 11.41, 10.68, 10.33, 10.33, 10.19, 8.19, 8.19, 7.01, 6.07, 5.51, 4.94, 4.77, 4.71, 4.29, 4.24, 3.91, 3.72, 3.66, 2.51, 1.7, 1.52, 1.22, 1.13, 0.75, 0.6, 0.47, 0.31, 0.12]
[2025-03-20 17:55:35]  there are 34 items in inventory with total value: 268.62
[2025-03-20 17:59:09]  bet_value = 2.27 -> bet_items_valuebet_items_values=[1.52, 0.75]
[2025-03-20 17:59:15]  coinflip_value=2.51 -> coinflip_items_values=[1.38, 1.13]
[2025-03-20 18:00:20]  Coin landed on: Black
[2025-03-20 18:00:20]  flip was won!
[2025-03-20 18:00:40]  there are 36 items in inventory with total value: 271.13
[2025-03-20 18:00:41]  Back to searching
[2025-03-20 18:00:52]  bet_value = 3.07 -> bet_items_valuebet_items_values=[1.22, 0.47, 1.38]
[2025-03-20 18:00:58]  coinflip_value=3.39 -> coinflip_items_values=[3.39]
[2025-03-20 18:01:17]  Couldn't determine coin face. Retry getting flipping face
[2025-03-20 18:01:17]  Coin landed on

In [14]:
driver.switch_to.window(original_window)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: MicrosoftEdge=134.0.3124.72)
Stacktrace:
	GetHandleVerifier [0x00A20E93+39155]
	Microsoft::Applications::Events::time_ticks_t::time_ticks_t [0x008BE3B6+772070]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x006748EE+5182]
	Microsoft::Applications::Events::DebugEventListener::~DebugEventListener [0x0065CDC5+49621]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x006C8A01+232705]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x006CE9DB+257243]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x006C1E43+205123]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x006A2947+76871]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x006A1CF5+73717]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x006A2764+76388]
	sqlite3_dbdata_init [0x00B265BC+589708]
	Microsoft::Applications::Events::FromJSON [0x00BF0473+699091]
	Microsoft::Applic

In [59]:
get_winner()

[2025-03-20 13:39:41]  Coin landed on: Red


('Red', 'Red')